In [1]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

# Importing Gensim
import gensim
from gensim import corpora

import sys
reload(sys)
sys.setdefaultencoding('utf-8')

stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
doc_complete = []

def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

Read data into a DataFrame

In [2]:
data = pd.read_json('data.json')

This will show all the column names

In [3]:
data.columns

Index([u'@context', u'@type', u'conformsTo', u'dataset', u'describedBy'], dtype='object')

Sample NASA's data.json

In [4]:
data.head()

,@context,@type,conformsTo,dataset,describedBy
0,https://project-open-data.cio.gov/v1.1/schema/...,dcat:Catalog,https://project-open-data.cio.gov/v1.1/schema,{u'publisher': {u'subOrganizationOf': {u'subOr...,https://project-open-data.cio.gov/v1.1/schema/...
1,https://project-open-data.cio.gov/v1.1/schema/...,dcat:Catalog,https://project-open-data.cio.gov/v1.1/schema,"{u'issued': u'2008-12-02T00:00:00.000Z', u'lic...",https://project-open-data.cio.gov/v1.1/schema/...
2,https://project-open-data.cio.gov/v1.1/schema/...,dcat:Catalog,https://project-open-data.cio.gov/v1.1/schema,"{u'issued': u'2008-12-02T00:00:00.000Z', u'ref...",https://project-open-data.cio.gov/v1.1/schema/...
3,https://project-open-data.cio.gov/v1.1/schema/...,dcat:Catalog,https://project-open-data.cio.gov/v1.1/schema,"{u'issued': u'2012-12-12T00:00:00.000Z', u'key...",https://project-open-data.cio.gov/v1.1/schema/...
4,https://project-open-data.cio.gov/v1.1/schema/...,dcat:Catalog,https://project-open-data.cio.gov/v1.1/schema,"{u'issued': u'2012-12-12T00:00:00.000Z', u'key...",https://project-open-data.cio.gov/v1.1/schema/...


In [5]:
file = open("doc_complete.txt", "w")

for result in data['dataset']:
    result[u'description'] = result[u'description']
    doc_complete.append(result[u'description'])
    
file.write(str(doc_complete))

In [6]:
doc_clean = [clean(doc).split() for doc in doc_complete]  

In [ ]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [ ]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=8, id2word = dictionary, passes=50)

In [ ]:
print(ldamodel.print_topics(num_topics=8, num_words=3))